In [1]:
import copy
import collections

from utils import read_json_to_dict
from PetReader import pet_reader
from petreader.labels import *

INFO:PetReader:Load RelationsExtraction dataset ...




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

INFO:PetReader:Load TokenClassification dataset ...




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

## Load Results

In [6]:
token_cls_goldstandard = read_json_to_dict("data/other/token_goldstandard.json")

keywords_gold_token_cls_results = read_json_to_dict("data/results/key_words_gold/results-token-classification.json")
keywords_gold_token_cls = read_json_to_dict("data/results/key_words_gold/token-classification.json")

keywords_literature_token_cls_results = read_json_to_dict("data/results/key_words_literature/results-token-classification.json")
keywords_literature_token_cls = read_json_to_dict("data/results/key_words_literature/token-classification.json")

## Debug

In [3]:
from petreader.TokenClassification import TokenClassification
from petbenchmarks.tokenclassification import TokenClassificationBenchmark
benchmark = TokenClassificationBenchmark(pet_reader.token_dataset)

gold = benchmark.GetGoldStandard()
print(gold['doc-3.2'][XOR_GATEWAY])

['If', 'otherwise']


In [4]:
print(pet_reader.token_dataset.GetXORGateways('doc-3.2'))

[[], [['If'], ['otherwise']], [], []]


## 1) Analyze Gold Keyword Results

### a) check for documents with recall != 1

In [5]:
for doc_name, doc_dict in keywords_gold_token_cls_results.items():
    if doc_name.startswith('doc'):
        if doc_dict[XOR_GATEWAY][RECALL] != 1 and doc_dict[XOR_GATEWAY][SUPPORT] != 0:
            print(doc_name.center(100, '-'))
            print("--Text--")
            for i, line in enumerate(pet_reader.get_doc_sentences(doc_name)):
                print(i, ' '.join(line))
            print()
            print("--Results--:", doc_dict[XOR_GATEWAY])
            print()
            print("--Extracted--:", keywords_gold_token_cls[doc_name][XOR_GATEWAY])
            print()
            print("--Gold standard--:", token_cls_goldstandard[doc_name][XOR_GATEWAY])
            print()

### b) Analyze False Positives

In [25]:
def analyze_keyword_stats(gold_standard, extracted_tokens, gateway_type: str, doc_names=[]):
    keyword_stats = {}
    empty_stats_dict = {"TP": 0, "FP": 0, "FN": 0, "TPs": [], "FPs": [], "FNs": []}

    for doc_name, doc_dict in keywords_gold_token_cls_results.items():
        
        if doc_name.startswith('doc'):  # result dict contains as well keys for statistics            
            if not doc_names or doc_name in doc_names:  # filter for (optionally) passed list

                # remove gateways step by step if they were identified; left overs are false positives
                not_discovered_golds = [g.lower() for g in gold_standard[doc_name][gateway_type].copy()]

                for extracted in extracted_tokens[doc_name][gateway_type]:
                    extracted = extracted.lower()

                    # setup keyword dict in case it was not observed yet
                    if extracted not in keyword_stats:
                        keyword_stats[extracted] = copy.deepcopy(empty_stats_dict)

                    # 1) CHECK FOR FPs
                    if extracted not in not_discovered_golds:
                        keyword_stats[extracted]["FP"] += 1
                        keyword_stats[extracted]["FPs"].append(doc_name)

                    # 2) CHECK FOR TPs
                    else:
                        keyword_stats[extracted]["TP"] += 1
                        keyword_stats[extracted]["TPs"].append(doc_name)
                        not_discovered_golds.remove(extracted)

                # 3) FILL FNs (FPs from list not_discovered_xor_golds got removed during previous loop)
                for not_extracted in not_discovered_golds:

                    # setup keyword dict in case it was not observed yet
                    if not_extracted not in keyword_stats:
                        keyword_stats[not_extracted] = copy.deepcopy(empty_stats_dict)

                    keyword_stats[not_extracted]["FN"] += 1
                    keyword_stats[not_extracted]["FNs"].append(doc_name)
    return keyword_stats


def print_stats(keyword_stats, first_x=None, order_by=None):
    
    if order_by:
        keyword_stats = collections.OrderedDict(sorted(keyword_stats.items(), key=lambda kv_pair: kv_pair[1][order_by],
                                                      reverse=True))
    for keyword, stats in keyword_stats.items():
        print(f"{keyword} ".ljust(15), end='')
        for k, v in stats.items():
            print(f"{k}: ", end='')
            if k.endswith("s") and first_x:
                print(v[:first_x], "...", end='')
            else:
                print(v, end='')
            print(", ", end='')
        print()


gold_xor_keyword_stats = analyze_keyword_stats(token_cls_goldstandard, keywords_gold_token_cls, XOR_GATEWAY)
gold_and_keyword_stats = analyze_keyword_stats(token_cls_goldstandard, keywords_gold_token_cls, AND_GATEWAY)
literature_xor_keyword_stats = analyze_keyword_stats(token_cls_goldstandard, keywords_literature_token_cls, XOR_GATEWAY)
literature_and_keyword_stats = analyze_keyword_stats(token_cls_goldstandard, keywords_literature_token_cls, AND_GATEWAY)

#### EXCLUSIVE

In [26]:
# GOLD keywords
print_stats(gold_xor_keyword_stats, first_x=2, order_by="FP")

for            TP: 6, FP: 63, FN: 0, TPs: ['doc-2.1', 'doc-2.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
or             TP: 20, FP: 44, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
if             TP: 61, FP: 6, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: ['doc-2.1', 'doc-10.10'] ..., FNs: [] ..., 
should         TP: 2, FP: 6, FN: 0, TPs: ['doc-6.1', 'doc-6.1'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
either         TP: 1, FP: 4, FN: 0, TPs: ['doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.1'] ..., FNs: [] ..., 
in             TP: 9, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
case           TP: 12, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
of             TP: 5, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
otherwise      TP: 12, FP: 1, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: [

In [28]:
# LITERATURE keywords
print_stats(literature_xor_keyword_stats, first_x=2, order_by="FP")

or             TP: 20, FP: 44, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
when           TP: 0, FP: 15, FN: 0, TPs: [] ..., FPs: ['doc-3.6', 'doc-6.2'] ..., FNs: [] ..., 
whether        TP: 0, FP: 13, FN: 0, TPs: [] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
if             TP: 61, FP: 6, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: ['doc-2.1', 'doc-10.10'] ..., FNs: [] ..., 
either         TP: 1, FP: 4, FN: 0, TPs: ['doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.1'] ..., FNs: [] ..., 
in             TP: 9, FP: 2, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: [] ..., 
case           TP: 9, FP: 2, FN: 3, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: ['doc-2.1', 'doc-2.1'] ..., 
of             TP: 5, FP: 2, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: [] ..., 
only           TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-2.2', 'doc-4.1'] ..., FNs: [] ..., 


#### PARALLEL

In [30]:
# GOLD keywords
print_stats(gold_and_keyword_stats, first_x=2, order_by="FP")

whereas        TP: 1, FP: 1, FN: 0, TPs: ['doc-1.2'] ..., FPs: ['doc-9.5'] ..., FNs: [] ..., 
at             TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
the            TP: 2, FP: 0, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
same           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
time           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
in             TP: 1, FP: 0, FN: 0, TPs: ['doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
meantime       TP: 3, FP: 0, FN: 0, TPs: ['doc-1.1', 'doc-3.5'] ..., FPs: [] ..., FNs: [] ..., 
two            TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
concurrent     TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
activities     TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
are            TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
triggered      TP: 1, FP: 0, FN: 0

In [29]:
# LITERATURE keywords
print_stats(literature_and_keyword_stats, first_x=2, order_by="FP")

in             TP: 1, FP: 2, FN: 0, TPs: ['doc-1.1'] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
addition       TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
to             TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
whereas        TP: 1, FP: 1, FN: 0, TPs: ['doc-1.2'] ..., FPs: ['doc-9.5'] ..., FNs: [] ..., 
at             TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
the            TP: 2, FP: 0, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
same           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
time           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
meantime       TP: 3, FP: 0, FN: 0, TPs: ['doc-1.1', 'doc-3.5'] ..., FPs: [] ..., FNs: [] ..., 
two            TP: 0, FP: 0, FN: 1, TPs: [] ..., FPs: [] ..., FNs: ['doc-2.1'] ..., 
concurrent     TP: 0, FP: 0, FN: 1, TPs: [] ..., FPs: [] ..., FNs: ['doc-2.1'